In [122]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import mean_squared_error

In [123]:
data = pd.read_csv("../../Downloads/AOD_series_IPW.csv")
data

,cell,week,aod,elev,temp,mslp
0,1,Wk_1_2014,nan,nan,nan,nan
1,1,Wk_2_2014,nan,nan,nan,nan
2,1,Wk_3_2014,nan,nan,nan,nan
3,1,Wk_4_2014,nan,nan,nan,nan
4,1,Wk_5_2014,nan,nan,nan,nan
...,...,...,...,...,...,...
15575995,59000,Wk_49_2018,nan,nan,nan,nan
15575996,59000,Wk_50_2018,nan,nan,nan,nan
15575997,59000,Wk_51_2018,nan,nan,nan,nan
15575998,59000,Wk_52_2018,nan,nan,nan,nan


In [124]:
data_cull = pd.DataFrame(data[~np.isnan(data["temp"])], copy=True)

In [125]:
data_cull["exists"] = -np.isnan(data_cull["aod"]).astype(int) + 1
data_cull

,cell,week,aod,elev,temp,mslp,exists
111408,423,Wk_1_2014,nan,265.1925,10.0669,1022.4109,0
111409,423,Wk_2_2014,nan,265.1925,10.0815,1018.2041,0
111410,423,Wk_3_2014,nan,265.1925,21.1717,1007.0175,0
111411,423,Wk_4_2014,nan,265.1925,8.7543,1015.0508,0
111412,423,Wk_5_2014,nan,265.1925,8.2720,1019.0836,0
...,...,...,...,...,...,...,...
15561475,58945,Wk_49_2018,nan,223.7122,33.9530,1010.4134,0
15561476,58945,Wk_50_2018,nan,223.7122,27.3003,1025.0060,0
15561477,58945,Wk_51_2018,nan,223.7122,34.0480,1019.8250,0
15561478,58945,Wk_52_2018,nan,223.7122,35.3497,1014.0676,0


In [126]:
X = data_cull[["elev", "temp", "mslp"]]
y = data_cull["exists"]

In [6]:
ipw_model = LogisticRegression(random_state = 1312).fit(X, y)

In [7]:
mean_absolute_error(y, ipw_model.predict_proba(X)[:, 1])

0.3829273097032682

In [8]:
ipw_model.classes_

array([0, 1], dtype=int64)

In [9]:
ipw = 1 / ipw_model.predict_proba(X)[:, 1]
ipw

array([5.32604128, 5.29728399, 3.4146381 , ..., 2.18013573, 2.09603664,
       2.17449365])

Now, adding a model that considers week.

In [127]:
week = np.char.partition(np.char.partition(np.array(data_cull["week"], str), sep="_")[:, 2], sep="_")[:, 0]
year = np.char.partition(np.char.partition(np.array(data_cull["week"], str), sep="_")[:, 2], sep="_")[:, 2]
year = year.astype(int) - 2000

In [11]:
X = pd.concat([data_cull[["elev", "temp", "mslp"]].reset_index(drop=True), pd.Series(week)], axis=1)
y = data_cull["exists"]

In [12]:
X.index = data_cull.index

In [13]:
ipw_model = LogisticRegression(random_state = 1312).fit(X, y)

In [14]:
mean_absolute_error(y, ipw_model.predict_proba(X)[:, 1])

0.38244091297258237

Results are slightly better; as this model is in line with the literature we'll keep it regardless.

In [15]:
ipw_model.classes_

array([0, 1], dtype=int64)

In [16]:
ipw = 1 / ipw_model.predict_proba(X)[:, 1]
ipw

array([5.04411525, 5.03525883, 3.24770256, ..., 2.32838869, 2.23740766,
       2.33302117])

For now, we need this model to be monthly.

In [128]:
data_cull["month"] = (week.astype(int) / (53 / 12)).astype(int).astype(str)
data_cull["year"] = year
data_month = data_cull.groupby(["month", "year", "cell"]).mean().reset_index()
data_month["exists"] = -np.isnan(data_month["aod"]).astype(int) + 1
data_month

,month,year,cell,aod,elev,temp,mslp,exists
0,0,14,423,nan,265.1925,12.5186,1015.6708,0
1,0,14,424,nan,268.0617,12.5293,1015.6703,0
2,0,14,425,nan,267.3794,12.5397,1015.6697,0
3,0,14,426,nan,267.5515,12.5496,1015.6692,0
4,0,14,427,nan,273.1963,12.5593,1015.6686,0
...,...,...,...,...,...,...,...,...
1992505,9,18,58941,0.1805,227.2763,54.8648,1018.4368,1
1992506,9,18,58942,0.2148,227.5434,54.8656,1018.4365,1
1992507,9,18,58943,0.1422,226.3096,54.8665,1018.4363,1
1992508,9,18,58944,0.1511,224.3239,54.8673,1018.4361,1


In [129]:
X = data_month[["elev", "temp", "mslp", "month"]]
y = data_month["exists"]

In [130]:
ipw_model = LogisticRegression(random_state = 1312).fit(X, y)

In [131]:
mean_absolute_error(y, ipw_model.predict_proba(X)[:, 1])

0.18401509148112435

Now, we'll pull in data for the old neural net, and create IPW weights.

In [132]:
df = pd.read_csv("../../Downloads/first_stage_20210601.csv")
df

,Raster.Cell,Sen2.5,month,year,AOD,Temp,Elev,MSLP,Vsby,WdVl,...,LC_LowDev,LC_HighDev,PS,relh,Popd,x,y,primary,secondary,motorway
0,3798,14.3000,3,14,0.1026,27.2144,202.6511,1018.8516,8.4725,8.4067,...,0.2470,8.4067,13.1464,68.5915,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493
1,3798,6.8000,4,14,0.0624,42.5520,202.6511,1014.2213,8.7911,8.7160,...,0.2470,8.7160,13.1464,67.1438,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493
2,3798,5.6000,5,14,0.1815,55.1383,202.6511,1015.4208,9.1797,7.0120,...,0.2470,7.0120,13.1464,68.5747,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493
3,3798,7.5714,6,14,0.1658,63.7574,202.6511,1013.5480,7.3425,6.3811,...,0.2470,6.3811,13.1464,74.5546,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493
4,3798,8.4714,7,14,0.2997,66.0770,202.6511,1015.3088,9.4635,6.0967,...,0.2470,6.0967,13.1464,69.6442,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,46895,6.6492,6,18,0.1504,71.8913,180.4956,1012.8235,8.9683,5.5281,...,0.0000,5.5281,9.7356,77.8714,0.0000,-88.1858,41.2243,0.0000,0.0000,0.0000
1491,46895,7.2063,7,18,0.2035,73.6890,180.4956,1017.0145,9.6198,4.5049,...,0.0000,4.5049,9.7356,74.2394,0.0000,-88.1858,41.2243,0.0000,0.0000,0.0000
1492,46895,9.5701,8,18,0.2873,73.9405,180.4956,1015.0842,9.0310,4.5487,...,0.0000,4.5487,9.7356,77.6731,0.0000,-88.1858,41.2243,0.0000,0.0000,0.0000
1493,46895,5.2483,10,18,0.1025,52.0867,180.4956,1017.4701,9.2085,6.4142,...,0.0000,6.4142,9.7356,75.7757,0.0000,-88.1858,41.2243,0.0000,0.0000,0.0000


In [133]:
# Remove the outliers
df=df[df['Sen2.5']<20].reset_index()
df = df.drop(["index"],axis=1)

In [134]:
ipw_weights = 1 / ipw_model.predict_proba(df[["Elev", "Temp", "MSLP", "month"]])[:, 1]
ipw_weights

array([1.50436348, 1.06891261, 1.01377333, ..., 1.00125491, 1.02824869,
       1.44869803])

Now, replicate the scikit-learn MLPRegressor framework in keras.

In [135]:
# transform data as in original nn
df2 = df.join(pd.get_dummies(df['month'],drop_first=True)).join(pd.get_dummies(df['year'],drop_first=True))
df2 = df2.drop(["month","year"], axis=1)

In [24]:
stopper = EarlyStopping(monitor="loss", patience=10, restore_best_weights=True)

In [160]:
mlp = Sequential()
mlp.add(Dense(10, activation="relu"))
mlp.add(Dense(10, activation="relu"))
mlp.add(Dense(1))
# note: batches default to 200

In [161]:
mlp.compile(loss="mean_squared_error", optimizer="adam")
mlp.fit(df2.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1), df2['Sen2.5'], epochs=2000, batch_size=200, callbacks=[stopper])

Epoch 1/2000
8/8 [==============================] - 0s 932us/step - loss: 343772.7188
Epoch 2/2000
8/8 [==============================] - 0s 1ms/step - loss: 228603.4375
Epoch 3/2000
8/8 [==============================] - 0s 1ms/step - loss: 154059.8281
Epoch 4/2000
8/8 [==============================] - 0s 1ms/step - loss: 103130.3359
Epoch 5/2000
8/8 [==============================] - 0s 1ms/step - loss: 68746.7969
Epoch 6/2000
8/8 [==============================] - 0s 909us/step - loss: 47792.1914
Epoch 7/2000
8/8 [==============================] - 0s 1ms/step - loss: 34673.0781
Epoch 8/2000
8/8 [==============================] - 0s 820us/step - loss: 25025.1953
Epoch 9/2000
8/8 [==============================] - 0s 1ms/step - loss: 18770.8340
Epoch 10/2000
8/8 [==============================] - 0s 1ms/step - loss: 14444.6338
Epoch 11/2000
8/8 [==============================] - 0s 973us/step - loss: 11415.7939
Epoch 12/2000
8/8 [==============================] - 0s 1ms/step - loss: 

Fitting seems reasonable; some changes, like removing outliers, scaling variables are missing, so needs work yet to match the base model. We will do that work as we implement the IPW loss function.

In [136]:
def weighted_mse(y_true, y_pred, weight):
    return weight * mean_squared_error(y_true, y_pred)

In [26]:
def get_model():
    """
    resets the neural net model for each fold
    """
    # input tensors; the last 2 are psuedo-inputs to use for the loss
    mlp = Input(shape=(31,))
    y_true = Input(shape=(1,))
    weight = Input(shape=(1,))
    
    # here we use the Model framework, instead of Sequential, as the model has multiple pseudo-inputs
    x = Dense(10, activation="relu")(mlp)
    x = Dense(10, activation="relu")(x)
    y_pred = Dense(1)(mlp)
    model = Model(inputs=[mlp, y_true, weight], outputs=y_pred)
    model.add_loss(weighted_mse(y_true, y_pred, weight))
    model.compile(loss=None, optimizer="adam")
    return model

In [194]:
model = get_model()
stopper = EarlyStopping(monitor="loss", patience=10, restore_best_weights=True)
model.fit([df2.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1), df2['Sen2.5'], ipw_weights], epochs=2000, batch_size=200, callbacks=[stopper])

Epoch 1/2000
8/8 [==============================] - 0s 836us/step - loss: 396921.9688
Epoch 2/2000
8/8 [==============================] - 0s 593us/step - loss: 303017.1875
Epoch 3/2000
8/8 [==============================] - 0s 574us/step - loss: 231860.5312
Epoch 4/2000
8/8 [==============================] - 0s 606us/step - loss: 170617.1875
Epoch 5/2000
8/8 [==============================] - 0s 948us/step - loss: 129357.6094
Epoch 6/2000
8/8 [==============================] - 0s 703us/step - loss: 90878.5938
Epoch 7/2000
8/8 [==============================] - 0s 816us/step - loss: 68344.0156
Epoch 8/2000
8/8 [==============================] - 0s 973us/step - loss: 49734.0352
Epoch 9/2000
8/8 [==============================] - 0s 1ms/step - loss: 37760.5234
Epoch 10/2000
8/8 [==============================] - 0s 1ms/step - loss: 29234.6035
Epoch 11/2000
8/8 [==============================] - 0s 884us/step - loss: 23506.3984
Epoch 12/2000
8/8 [==============================] - 0s 1ms/st

In [195]:
# this model copies weights and removes the other 2 inputs for easier use
# as shown below, this does not appear to actually work??
pred_model = Model(inputs=mlp, outputs=y_pred)

In [196]:
pred_model.predict(df2.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1))

array([[2389.9194  ],
       [2389.7512  ],
       [2389.8975  ],
       ...,
       [  98.87438 ],
       [  98.93127 ],
       [  99.875916]], dtype=float32)

In [197]:
model.predict([df2.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1), df2['Sen2.5'], ipw_weights])

array([[13.507674],
       [12.085395],
       [10.660815],
       ...,
       [ 9.463694],
       [12.019959],
       [10.612422]], dtype=float32)

Let's put it all together.

In [33]:
#Train the neural network model
# random CV
Xvars = df2.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1)
yvars = df2['Sen2.5']
weights = pd.Series(ipw_weights)
weights.index = Xvars.index
model_weights = []

kf = KFold(n_splits=10, random_state=10, shuffle=True)

MSE_vec_kf = np.zeros(10)
r2_vec_kf_train =  np.zeros(10)
r2_vec_kf = np.zeros(10)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars.loc[train_index], Xvars.loc[test_index]
    y_train, y_test = yvars.loc[train_index], yvars.loc[test_index]
    w_train, w_test = weights.loc[train_index], weights.loc[test_index]

    # Standarize the input variables
    scaler = MinMaxScaler()
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # split again for validation data
    X_t, X_val, y_t, y_val, w_t, w_val = train_test_split(X_train, y_train, w_train)
    
    stopper = EarlyStopping(patience=10, restore_best_weights=True)
    mlp = get_model()
    # TODO: need to work this; stopper is too greedy/training too volatile
    # TODO: also consider transform, inverse-transform for y vars
    mlp.fit([X_t, y_t, w_t], validation_data=[X_val, y_val, w_val], epochs=2000, batch_size=200, callbacks=[stopper], verbose=0)
    y_pred_train = mlp.predict([X_train, y_train, w_train])
    y_pred = mlp.predict([X_test, y_test, w_test]).reshape(len(test_index))
    r2_vec_kf_train[k_ind] =  r2_score(y_train, y_pred_train)
    r2_vec_kf[k_ind] = r2_score(y_test, y_pred)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('r2 for test set', k_ind, ' is', r2_vec_kf[k_ind])
    k_ind += 1
    model_weights.append(mlp.get_weights())

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)

r2_kf =r2_vec_kf.mean()
r2_kf_std = r2_vec_kf.std()
print('test estimate r2 k-fold=', r2_kf,
      'test estimate r2 standard err=', r2_kf_std)

r2_kf_train =r2_vec_kf_train.mean()
r2_kf_std_train = r2_vec_kf_train.std()
print('train set r2 k-fold=', r2_kf_train,
      'train set r2 standard err=', r2_kf_std_train)

MSE for test set 0  is 5.015851518727484
r2 for test set 0  is 0.4407873320870841
MSE for test set 1  is 6.820429579461351
r2 for test set 1  is 0.4117497881137029
MSE for test set 2  is 18.519281446340596
r2 for test set 2  is 0.21556304935740644
MSE for test set 3  is 6.984029665803567
r2 for test set 3  is 0.37480548612185516
MSE for test set 4  is 3.4230288744993596
r2 for test set 4  is 0.32954060369044036
MSE for test set 5  is 2.997027618890876
r2 for test set 5  is 0.43930214104604925
MSE for test set 6  is 3.498024343140673
r2 for test set 6  is 0.31235751463990746
MSE for test set 7  is 3.936440877843625
r2 for test set 7  is 0.23936545285222888
MSE for test set 8  is 4.223046426437252
r2 for test set 8  is 0.3548824577458908
MSE for test set 9  is 3.440532775524327
r2 for test set 9  is 0.3802306926213215
test estimate MSE k-fold= 5.885769312666911 test estimate MSE standard err= 4.416979816140345
test estimate r2 k-fold= 0.34985845182758873 test estimate r2 standard err= 0.

In [34]:
# average the weights of the 10 models
mean_weights = [np.full(model_weights[0][0].shape, 0.0), np.full(model_weights[0][1].shape, 0.0)]
for weight in model_weights:
    for i in range(len(weight)):
        mean_weights[i] += weight[i] / len(model_weights)

In [51]:
model_mean = get_model()
model_mean.set_weights(mean_weights)

In [52]:
y_pred = model_mean.predict([scaler.transform(Xvars), yvars, weights]).reshape(len(yvars))
y_test = yvars
print(r2_score(y_test, y_pred))
print(((y_test - y_pred) ** 2).mean())

0.37672402177127207
5.549890946624801


In [53]:
stats = pd.DataFrame(np.array(["MSE", ((y_test - y_pred) ** 2).mean(), MSE_kf_std, "R2", r2_score(y_test, y_pred), r2_kf_std]).reshape(2, 3),\
            columns=["statistic", "mean", "sd"])
stats

,statistic,mean,sd
0,MSE,5.549890946624801,4.433157477549257
1,R2,0.37672402177127207,0.07100513887145664


In [55]:
stats.to_csv("../../Downloads/nn_outlier_stats_ipw.csv")

### Spatial CV version

In [40]:
kMeansVars = df2.drop('Sen2.5', axis=1)
Xvars = df2.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1)
yvars = df2['Sen2.5']

In [41]:
kmeans = KMeans(10).fit(np.array(kMeansVars[["x", "y"]]).reshape(-1, 2))

In [42]:
# check fold sizes
fold_sizes = np.zeros(10)
for fold in np.unique(kmeans.labels_):
    fold_sizes[fold] = len(np.unique(df2[kmeans.labels_ == fold]["Raster.Cell"]))
    if (len(np.unique(df2[kmeans.labels_ == fold]["Raster.Cell"])) == 1):
        print(len(np.unique(df2[kmeans.labels_ == fold]["Raster.Cell"])), df2[kmeans.labels_ == fold]["Raster.Cell"].iloc[0])
        continue
    print(len(np.unique(df2[kmeans.labels_ == fold]["Raster.Cell"])))

1 4557
6
8
2
1 46895
5
1 35637
1 16743
3
4


In [44]:
#Train the neural network model

MSE_vec_kf = np.zeros(len(np.unique(df2["Raster.Cell"])))
r2_vec_kf_train =  np.zeros(len(np.unique(df2["Raster.Cell"])))
r2_vec_kf = np.zeros(len(np.unique(df2["Raster.Cell"])))
weights = pd.Series(ipw_weights)
weights.index = Xvars.index
model_weights = []

k_ind = int(0)
for fold in np.unique(kmeans.labels_):
    X_train, X_test = Xvars.loc[kmeans.labels_ != fold], Xvars.loc[kmeans.labels_ == fold]
    y_train, y_test = yvars.loc[kmeans.labels_ != fold], yvars.loc[kmeans.labels_ == fold]
    w_train, w_test = weights.loc[kmeans.labels_ != fold], weights.loc[kmeans.labels_ == fold]

    # Standarize the input variables
    scaler = MinMaxScaler()
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # split again for validation data
    X_t, X_val, y_t, y_val, w_t, w_val = train_test_split(X_train, y_train, w_train)
    
    stopper = EarlyStopping(patience=10, restore_best_weights=True)
    mlp = get_model()
    # TODO: need to work this; stopper is too greedy/training too volatile
    # TODO: also consider transform, inverse-transform for y vars
    mlp.fit([X_t, y_t, w_t], validation_data=[X_val, y_val, w_val], epochs=2000, batch_size=200, callbacks=[stopper], verbose=0)
    y_pred_train = mlp.predict([X_train, y_train, w_train])
    y_pred = mlp.predict([X_test, y_test, w_test]).reshape((kmeans.labels_ == fold).sum())
    r2_train =  r2_score(y_train, y_pred_train)
    r2 = r2_score(y_test, y_pred)
    MSE = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', fold, ' is', MSE)
    print('r2 for test set', fold, ' is', r2)
    model_weights.append(mlp.get_weights())
    
    # repeats the test statistic values to weight by fold size
    for idx in range(int(fold_sizes[fold])):
        MSE_vec_kf[idx + k_ind] = MSE
        r2_vec_kf[idx + k_ind] = r2
        r2_vec_kf_train[idx + k_ind] = r2_train
    
    k_ind += int(fold_sizes[fold])
    
MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)

r2_kf =r2_vec_kf.mean()
r2_kf_std = r2_vec_kf.std()
print('test estimate r2 k-fold=', r2_kf,
      'test estimate r2 standard err=', r2_kf_std)

r2_kf_train =r2_vec_kf_train.mean()
r2_kf_std_train = r2_vec_kf_train.std()
print('train set r2 k-fold=', r2_kf_train,
      'train set r2 standard err=', r2_kf_std_train)

MSE for test set 0  is 4.945038355872958
r2 for test set 0  is -0.16197628508210093
MSE for test set 1  is 4.9596424518673645
r2 for test set 1  is 0.1370923489855732
MSE for test set 2  is 7.580681274142173
r2 for test set 2  is -0.36840289329606546
MSE for test set 3  is 3.1790099078559657
r2 for test set 3  is 0.4830218291913334
MSE for test set 4  is 3.41797565279036
r2 for test set 4  is 0.07062617382948677
MSE for test set 5  is 2.6837701310020234
r2 for test set 5  is 0.38410560242200675
MSE for test set 6  is 1.8813470315672658
r2 for test set 6  is 0.4705373800599264
MSE for test set 7  is 1.9265503697658717
r2 for test set 7  is 0.4854130243486757
MSE for test set 8  is 2.6244579582163707
r2 for test set 8  is 0.3987063049496824
MSE for test set 9  is 6.672437703007545
r2 for test set 9  is -0.5608388328387433
test estimate MSE k-fold= 4.903569108491856 test estimate MSE standard err= 2.0634001813283738
test estimate r2 k-fold= 0.01810207795898995 test estimate r2 standard er

In [45]:
# average the weights of the 10 models
mean_weights = [np.full(model_weights[0][0].shape, 0.0), np.full(model_weights[0][1].shape, 0.0)]
for weight in model_weights:
    for i in range(len(weight)):
        mean_weights[i] += weight[i] / len(model_weights)

In [46]:
model_mean = get_model()
model_mean.set_weights(mean_weights)

In [47]:
y_pred = model_mean.predict([scaler.transform(Xvars), yvars, weights]).reshape(len(yvars))
y_test = yvars
print(r2_score(y_test, y_pred))
print(((y_test - y_pred) ** 2).mean())

0.422527398249109
3.0899951036005397


In [49]:
stats = pd.DataFrame(np.array(["MSE", ((y_test - y_pred) ** 2).mean(), MSE_kf_std, "R2", r2_score(y_test, y_pred), r2_kf_std]).reshape(2, 3),\
            columns=["statistic", "mean", "sd"])
stats

,statistic,mean,sd
0,MSE,3.0899951036005397,2.0634001813283738
1,R2,0.422527398249109,0.3797889120155116


In [50]:
stats.to_csv("../../Downloads/nn_spatialcv_stats_ipw.csv")

### Stage 2

In [51]:
# Read in the second stage data
# df_second = pd.read_csv("NN_second_stage.csv")
df_second = pd.read_csv("../../Downloads/second_stage_20210601.csv")
df_second.head()

,Raster.Cell,Sen2.5,month,year,AOD,Temp,Elev,MSLP,Vsby,WdVl,...,LC_LowDev,LC_HighDev,PS,relh,Popd,x,y,primary,secondary,motorway
0,610,NaN,4,14,0.165171,42.344765,266.558136,1014.221252,8.944846,8.279146,...,0.0,8.279146,10.582934,68.651376,0.000033,-88.527968,43.190225,0.0,0.0,0.0
1,610,NaN,5,14,0.120292,55.061996,266.558136,1015.420776,9.219633,6.657452,...,0.0,6.657452,10.582934,70.345016,0.000033,-88.527968,43.190225,0.0,0.0,0.0
2,610,NaN,6,14,0.134082,63.962715,266.558136,1013.547974,7.662267,6.085362,...,0.0,6.085362,10.582934,76.186780,0.000033,-88.527968,43.190225,0.0,0.0,0.0
3,610,NaN,7,14,0.421659,65.683273,266.558136,1015.308838,9.537151,5.752195,...,0.0,5.752195,10.582934,72.986256,0.000033,-88.527968,43.190225,0.0,0.0,0.0
4,610,NaN,8,14,0.228383,67.505302,266.558136,1015.604980,8.546723,4.495462,...,0.0,4.495462,10.582934,80.359692,0.000033,-88.527968,43.190225,0.0,0.0,0.0


In [53]:
# Drop the cells vith NA values
df_second=df_second[(~df_second['NDVI'].isna())& (~df_second['Popd'].isna())]
df_second = df_second.drop("Sen2.5", axis=1)
df_second.shape

(1536690, 21)

In [54]:
# generate IPW weights for 2nd stage
ipw_weights_2 = 1 / ipw_model.predict_proba(df_second[["Elev", "Temp", "MSLP", "month"]])[:, 1]
ipw_weights_2

array([1.09792518, 1.01922982, 1.0062817 , ..., 1.03234357, 1.36567035,
       1.50930033])

In [55]:
cell = df_second[["Raster.Cell","month","year"]]

In [56]:
pd.options.display.float_format = "{:.4f}".format
df_second.drop(["Raster.Cell","month","year"], axis=1).describe()

,AOD,Temp,Elev,MSLP,Vsby,WdVl,NDVI,LC_MedDev,LC_LowDev,LC_HighDev,PS,relh,Popd,x,y,primary,secondary,motorway
count,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000,1536690.0000
mean,0.1663,53.8314,221.2532,1016.7156,8.8666,6.7942,0.4902,0.0953,0.0407,6.7942,12.1723,73.3439,0.0005,-87.8070,41.7993,723.4965,884.6209,599.1359
std,0.0938,15.3615,32.0532,2.6933,0.5477,1.4628,0.2027,0.1378,0.0805,1.4628,8.5076,5.5602,0.0012,0.5991,0.6392,1576.0544,1773.8034,2240.8581
min,0.0000,20.9046,149.3274,1011.2849,6.5153,2.9493,-0.1403,0.0000,0.0000,2.9493,1.0000,38.7312,0.0000,-88.7758,40.7412,0.0000,0.0000,0.0000
25%,0.1027,39.6843,199.5573,1014.7415,8.5675,5.6362,0.3217,0.0020,0.0000,5.6362,8.9648,69.7168,0.0000,-88.2802,41.2827,0.0000,0.0000,0.0000
50%,0.1425,56.1608,213.3082,1016.3637,8.9549,6.8828,0.4623,0.0210,0.0000,6.8828,11.2097,74.0631,0.0001,-87.9498,41.6325,0.0000,0.0000,0.0000
75%,0.2056,68.3399,239.6572,1018.6949,9.2581,7.8992,0.6532,0.1490,0.0390,7.8992,13.1616,77.1286,0.0004,-87.3598,42.3156,475.0318,1007.5490,0.0000
max,1.7891,75.3651,351.6488,1023.1313,9.8292,10.5263,0.9456,0.8350,0.5600,10.5263,382.0000,93.1307,0.0333,-86.4748,43.1902,18537.3689,23797.0901,40774.7948


In [57]:
df_2 = df_second
df_2['month_str']=df_2[["month"]].astype(str)
df_2['year_str'] =df_2[["year"]].astype(str)
df_2['time_str']= df_2['year_str']+'-'+df_2['month_str']

In [58]:
df_2 = df_2.join(pd.get_dummies(df_2['month'],drop_first=True)).join(pd.get_dummies(df_2['year'],drop_first=True))
df_2 = df_2.drop(['month_str','year_str','time_str'], axis=1)
df_2 = df_2.drop(["month","year","Raster.Cell","x","y"], axis=1)

In [59]:
# generate empty predictions (needed for structure)
y_2 = np.empty(df_2.shape[0])

In [60]:
# Use the model got from stage one to get the predictions
data = scaler.transform(df_2)
data_pred = model_mean.predict([data, y_2, ipw_weights_2])
df_2['pred'] = data_pred

In [61]:
df_2 = df_2.join(cell)

In [62]:
df_2=df_2.drop([2,3,4,5,6,7,8,9,10,11,12,15,16,17,18],axis=1)
df_2.head()

,AOD,Temp,Elev,MSLP,Vsby,WdVl,NDVI,LC_MedDev,LC_LowDev,LC_HighDev,PS,relh,Popd,primary,secondary,motorway,pred,Raster.Cell,month,year
0,0.1652,42.3448,266.5581,1014.2213,8.9448,8.2791,0.2914,0.0000,0.0000,8.2791,10.5829,68.6514,0.0000,0.0000,0.0000,0.0000,7.6992,610,4,14
1,0.1203,55.0620,266.5581,1015.4208,9.2196,6.6575,0.4495,0.0000,0.0000,6.6575,10.5829,70.3450,0.0000,0.0000,0.0000,0.0000,8.2122,610,5,14
2,0.1341,63.9627,266.5581,1013.5480,7.6623,6.0854,0.5470,0.0000,0.0000,6.0854,10.5829,76.1868,0.0000,0.0000,0.0000,0.0000,9.5130,610,6,14
3,0.4217,65.6833,266.5581,1015.3088,9.5372,5.7522,0.8203,0.0000,0.0000,5.7522,10.5829,72.9863,0.0000,0.0000,0.0000,0.0000,8.7368,610,7,14
4,0.2284,67.5053,266.5581,1015.6050,8.5467,4.4955,0.8328,0.0000,0.0000,4.4955,10.5829,80.3597,0.0000,0.0000,0.0000,0.0000,9.1163,610,8,14


In [63]:
# Merge the dataframe from stage one and stage two to get the data for stage three
df_3 = df
df_3['pred'] = df_3['Sen2.5']
df_3=df_3.drop('Sen2.5', axis=1)
df_3.head()

,Raster.Cell,month,year,AOD,Temp,Elev,MSLP,Vsby,WdVl,NDVI,...,LC_HighDev,PS,relh,Popd,x,y,primary,secondary,motorway,pred
0,3798,3,14,0.1026,27.2144,202.6511,1018.8516,8.4725,8.4067,0.1605,...,8.4067,13.1464,68.5915,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493,14.3000
1,3798,4,14,0.0624,42.5520,202.6511,1014.2213,8.7911,8.7160,0.2352,...,8.7160,13.1464,67.1438,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493,6.8000
2,3798,5,14,0.1815,55.1383,202.6511,1015.4208,9.1797,7.0120,0.3894,...,7.0120,13.1464,68.5747,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493,5.6000
3,3798,6,14,0.1658,63.7574,202.6511,1013.5480,7.3425,6.3811,0.4190,...,6.3811,13.1464,74.5546,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493,7.5714
4,3798,7,14,0.2997,66.0770,202.6511,1015.3088,9.4635,6.0967,0.4372,...,6.0967,13.1464,69.6442,0.0028,-87.9144,43.0569,83.3885,8613.3285,1269.4493,8.4714


In [64]:
df_4 = df_2.append(df_3, ignore_index=True)

In [65]:
df_4

,AOD,Temp,Elev,MSLP,Vsby,WdVl,NDVI,LC_MedDev,LC_LowDev,LC_HighDev,...,Popd,primary,secondary,motorway,pred,Raster.Cell,month,year,x,y
0,0.1652,42.3448,266.5581,1014.2213,8.9448,8.2791,0.2914,0.0000,0.0000,8.2791,...,0.0000,0.0000,0.0000,0.0000,7.6992,610,4,14,nan,nan
1,0.1203,55.0620,266.5581,1015.4208,9.2196,6.6575,0.4495,0.0000,0.0000,6.6575,...,0.0000,0.0000,0.0000,0.0000,8.2122,610,5,14,nan,nan
2,0.1341,63.9627,266.5581,1013.5480,7.6623,6.0854,0.5470,0.0000,0.0000,6.0854,...,0.0000,0.0000,0.0000,0.0000,9.5130,610,6,14,nan,nan
3,0.4217,65.6833,266.5581,1015.3088,9.5372,5.7522,0.8203,0.0000,0.0000,5.7522,...,0.0000,0.0000,0.0000,0.0000,8.7368,610,7,14,nan,nan
4,0.2284,67.5053,266.5581,1015.6050,8.5467,4.4955,0.8328,0.0000,0.0000,4.4955,...,0.0000,0.0000,0.0000,0.0000,9.1163,610,8,14,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538171,0.1504,71.8913,180.4956,1012.8235,8.9683,5.5281,0.7966,0.0300,0.0000,5.5281,...,0.0000,0.0000,0.0000,0.0000,6.6492,46895,6,18,-88.1858,41.2243
1538172,0.2035,73.6890,180.4956,1017.0145,9.6198,4.5049,0.7862,0.0300,0.0000,4.5049,...,0.0000,0.0000,0.0000,0.0000,7.2063,46895,7,18,-88.1858,41.2243
1538173,0.2873,73.9405,180.4956,1015.0842,9.0310,4.5487,0.6050,0.0300,0.0000,4.5487,...,0.0000,0.0000,0.0000,0.0000,9.5701,46895,8,18,-88.1858,41.2243
1538174,0.1025,52.0867,180.4956,1017.4701,9.2085,6.4142,0.3524,0.0300,0.0000,6.4142,...,0.0000,0.0000,0.0000,0.0000,5.2483,46895,10,18,-88.1858,41.2243


In [66]:
df_4.to_csv('../../Downloads/Third_stage_spatialcv_ipw.csv')

In [101]:
df_4.to_csv('../../Downloads/Third_stage_base_ipw.csv')

In [126]:
df_4.to_csv('../../Downloads/Third_stage_outlier_ipw.csv')

### Outlier testing

In [29]:
base_model = get_model()
base_model.set_weights(mean_weights)

In [35]:
outlier_model = get_model()
outlier_model.set_weights(mean_weights)

In [52]:
spatial_model = get_model()
spatial_model.set_weights(mean_weights)

In [146]:
df = pd.read_csv("../../Downloads/first_stage_20210601.csv")
ipw_weights = 1 / ipw_model.predict_proba(df[["Elev", "Temp", "MSLP", "month"]])[:, 1]
ipw_weights
# transform data as in original nn
df2 = df.join(pd.get_dummies(df['month'],drop_first=True)).join(pd.get_dummies(df['year'],drop_first=True))
df2 = df2.drop(["month","year"], axis=1)
df2

,Raster.Cell,Sen2.5,AOD,Temp,Elev,MSLP,Vsby,WdVl,NDVI,LC_MedDev,...,7,8,9,10,11,12,15,16,17,18
0,3798,14.3000,0.1026,27.2144,202.6511,1018.8516,8.4725,8.4067,0.1605,0.1770,...,0,0,0,0,0,0,0,0,0,0
1,3798,6.8000,0.0624,42.5520,202.6511,1014.2213,8.7911,8.7160,0.2352,0.1770,...,0,0,0,0,0,0,0,0,0,0
2,3798,5.6000,0.1815,55.1383,202.6511,1015.4208,9.1797,7.0120,0.3894,0.1770,...,0,0,0,0,0,0,0,0,0,0
3,3798,7.5714,0.1658,63.7574,202.6511,1013.5480,7.3425,6.3811,0.4190,0.1770,...,0,0,0,0,0,0,0,0,0,0
4,3798,8.4714,0.2997,66.0770,202.6511,1015.3088,9.4635,6.0967,0.4372,0.1770,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,46895,6.6492,0.1504,71.8913,180.4956,1012.8235,8.9683,5.5281,0.7966,0.0300,...,0,0,0,0,0,0,0,0,0,1
1491,46895,7.2063,0.2035,73.6890,180.4956,1017.0145,9.6198,4.5049,0.7862,0.0300,...,1,0,0,0,0,0,0,0,0,1
1492,46895,9.5701,0.2873,73.9405,180.4956,1015.0842,9.0310,4.5487,0.6050,0.0300,...,0,1,0,0,0,0,0,0,0,1
1493,46895,5.2483,0.1025,52.0867,180.4956,1017.4701,9.2085,6.4142,0.3524,0.0300,...,0,0,0,1,0,0,0,0,0,1


In [85]:
quantiles = np.quantile(df2["Sen2.5"], [0.10, 0.90])
quantiles

array([ 6.20826139, 12.12694874])

In [92]:
df_extreme = df2[(df2["Sen2.5"] < quantiles[0]) | (df2["Sen2.5"] > quantiles[1])]
df_hi = df2[df2["Sen2.5"] > quantiles[1]]
weights_extreme = ipw_weights[df_extreme.index]
weights_hi = ipw_weights[df_hi.index]

In [95]:
X_extreme = df_extreme.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1)
y_extreme = df_extreme['Sen2.5']
X_hi = df_hi.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1)
y_hi = df_hi['Sen2.5']

In [103]:
y_pred = base_model.predict([scaler.transform(X_extreme), y_extreme, weights_extreme]).reshape(len(y_extreme))
y_test = y_extreme
r2_base = r2_score(y_test, y_pred)
mse_base = ((y_test - y_pred) ** 2).mean()

In [115]:
y_pred = outlier_model.predict([scaler.transform(X_extreme), y_extreme, weights_extreme]).reshape(len(y_extreme))
y_test = y_extreme
r2_outlier = r2_score(y_test, y_pred)
mse_outlier = ((y_test - y_pred) ** 2).mean()

In [105]:
y_pred = spatial_model.predict([scaler.transform(X_extreme), y_extreme, weights_extreme]).reshape(len(y_extreme))
y_test = y_extreme
r2_spatial = r2_score(y_test, y_pred)
mse_spatial = ((y_test - y_pred) ** 2).mean()

In [106]:
y_pred = base_model.predict([scaler.transform(X_hi), y_hi, weights_hi]).reshape(len(y_hi))
y_test = y_hi
mse_base_hi = ((y_test - y_pred) ** 2).mean()

In [107]:
y_pred = outlier_model.predict([scaler.transform(X_hi), y_hi, weights_hi]).reshape(len(y_hi))
y_test = y_hi
mse_outlier_hi = ((y_test - y_pred) ** 2).mean()

In [108]:
y_pred = spatial_model.predict([scaler.transform(X_hi), y_hi, weights_hi]).reshape(len(y_hi))
y_test = y_hi
mse_spatial_hi = ((y_test - y_pred) ** 2).mean()

In [110]:
stats_extreme = pd.DataFrame.from_dict({"model": ["base", "outlier", "spatial"],\
                        "r2": [r2_base, r2_outlier, r2_spatial],\
                        "mse": [mse_base, mse_outlier, mse_spatial],\
                        "mse_hi": [mse_base_hi, mse_outlier_hi, mse_spatial_hi]})
stats_extreme

,model,r2,mse,mse_hi
0,base,0.3328,22.6477,41.7389
1,outlier,0.4252,19.5119,35.0932
2,spatial,0.3070,23.5234,43.9934


In [112]:
stats_extreme.to_csv("../../Downloads/nn_stats_extreme.csv")

In [147]:
# also, write original predictions to df
Xvars = df2.drop('Sen2.5', axis=1).drop(["Raster.Cell", "x", "y"], axis=1)
yvars = df2['Sen2.5']
y_pred_base = base_model.predict([scaler.transform(Xvars), yvars, ipw_weights]).reshape(len(yvars))
y_pred_outlier = outlier_model.predict([scaler.transform(Xvars), yvars, ipw_weights]).reshape(len(yvars))
y_pred_spatial = spatial_model.predict([scaler.transform(Xvars), yvars, ipw_weights]).reshape(len(yvars))
df2["pred_base"] = y_pred_base
df2["pred_outlier"] = y_pred_outlier
df2["pred_spatial"] = y_pred_spatial
cell = df[["month","year"]]
df2 = df2.join(cell)
df2 = df2.drop([2,3,4,5,6,7,8,9,10,11,12,15,16,17,18],axis=1)
df2

,Raster.Cell,Sen2.5,AOD,Temp,Elev,MSLP,Vsby,WdVl,NDVI,LC_MedDev,...,x,y,primary,secondary,motorway,pred_base,pred_outlier,pred_spatial,month,year
0,3798,14.3000,0.1026,27.2144,202.6511,1018.8516,8.4725,8.4067,0.1605,0.1770,...,-87.9144,43.0569,83.3885,8613.3285,1269.4493,8.5257,8.1240,8.2471,3,14
1,3798,6.8000,0.0624,42.5520,202.6511,1014.2213,8.7911,8.7160,0.2352,0.1770,...,-87.9144,43.0569,83.3885,8613.3285,1269.4493,7.0868,6.6625,6.8269,4,14
2,3798,5.6000,0.1815,55.1383,202.6511,1015.4208,9.1797,7.0120,0.3894,0.1770,...,-87.9144,43.0569,83.3885,8613.3285,1269.4493,8.1561,7.5928,7.9142,5,14
3,3798,7.5714,0.1658,63.7574,202.6511,1013.5480,7.3425,6.3811,0.4190,0.1770,...,-87.9144,43.0569,83.3885,8613.3285,1269.4493,10.0562,10.0779,9.8228,6,14
4,3798,8.4714,0.2997,66.0770,202.6511,1015.3088,9.4635,6.0967,0.4372,0.1770,...,-87.9144,43.0569,83.3885,8613.3285,1269.4493,9.8164,10.7443,9.5216,7,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,46895,6.6492,0.1504,71.8913,180.4956,1012.8235,8.9683,5.5281,0.7966,0.0300,...,-88.1858,41.2243,0.0000,0.0000,0.0000,6.4701,6.0915,6.1866,6,18
1491,46895,7.2063,0.2035,73.6890,180.4956,1017.0145,9.6198,4.5049,0.7862,0.0300,...,-88.1858,41.2243,0.0000,0.0000,0.0000,8.6466,9.4494,8.2950,7,18
1492,46895,9.5701,0.2873,73.9405,180.4956,1015.0842,9.0310,4.5487,0.6050,0.0300,...,-88.1858,41.2243,0.0000,0.0000,0.0000,9.5488,9.5434,9.2748,8,18
1493,46895,5.2483,0.1025,52.0867,180.4956,1017.4701,9.2085,6.4142,0.3524,0.0300,...,-88.1858,41.2243,0.0000,0.0000,0.0000,6.7939,6.8527,6.4612,10,18


In [149]:
print(np.mean((df2["pred_base"] - df2["Sen2.5"])**2))
print(np.mean((df2["pred_outlier"] - df2["Sen2.5"])**2))
print(np.mean((df2["pred_spatial"] - df2["Sen2.5"])**2))

5.858222138180245
5.558755549191473
6.123195048874126


In [148]:
df2.to_csv("../../Downloads/first_stage_w_pred.csv")